In [15]:
!pip install xgboost

In [110]:
!pip install shap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 533.5/533.5 kB 8.4 MB/s eta 0:00:00


In [16]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb

import pandas as pd
import numpy as np

In [ ]:
games = pd.read_csv("games.csv")
players = pd.read_csv("players.csv")
plays = pd.read_csv("plays.csv")
tackles = pd.read_csv("tackles.csv")

In [ ]:
all_players_sorted_1 = pd.read_pickle("all_players_sorted_1.pkl")
all_players_sorted_2 = pd.read_pickle("all_players_sorted_2.pkl")
all_players_sorted_3 = pd.read_pickle("all_players_sorted_3.pkl")
all_players_sorted_4 = pd.read_pickle("all_players_sorted_4.pkl")
all_players_sorted_5 = pd.read_pickle("all_players_sorted_5.pkl")
all_players_sorted_6 = pd.read_pickle("all_players_sorted_6.pkl")
all_players_sorted_7 = pd.read_pickle("all_players_sorted_7.pkl")
all_players_sorted_8 = pd.read_pickle("all_players_sorted_8.pkl")
all_players_sorted_9 = pd.read_pickle("all_players_sorted_9.pkl")

df_all = pd.read_pickle("df_all.pkl")

In [85]:
# splitting dataframe for games up to week 6, as well as those from weeks 7-9
df_all_1_6 = df_all[df_all['gameId'] < 2022101900]
df_all_7_9 = df_all[df_all['gameId'] > 2022101900]

In [86]:
df_reg_cols = ['max_distance', 'actual_dist_from_final', 'frame_since_bc','player1.0_y', 'player1.0_s', 'player1.0_dis_closest_o',
 'player1.0_ang_closest_o', 'player1.0_min_dist', 'player1.0_ang_min', 'player1.0_2_min_dist', 'player1.0_2_ang_min', 'player1.0_a',
 'player2.0_s', 'player2.0_dist_ball', 'player2.0_ang_ball', 'player2.0_dis_closest_o', 'player2.0_ang_closest_o', 'player2.0_min_dist',
 'player2.0_ang_min', 'player2.0_2_min_dist', 'player2.0_2_ang_min', 'player2.0_a', 'player3.0_s', 'player3.0_dist_ball', 'player3.0_ang_ball',
 'player3.0_dis_closest_o', 'player3.0_ang_closest_o', 'player3.0_min_dist', 'player3.0_ang_min', 'player3.0_2_min_dist', 'player3.0_2_ang_min',
 'player3.0_a', 'player4.0_s', 'player4.0_dist_ball', 'player4.0_ang_ball', 'player4.0_dis_closest_o', 'player4.0_ang_closest_o',
 'player4.0_min_dist', 'player4.0_ang_min', 'player4.0_2_min_dist', 'player4.0_2_ang_min', 'player4.0_a']

In [87]:
df_reg = df_all_1_6.loc[:, df_reg_cols]
df_reg_wks_7_9 = df_all_7_9.loc[:, df_reg_cols]

In [88]:
df_reg.head()

,max_distance,actual_dist_from_final,frame_since_bc,player1.0_y,player1.0_s,player1.0_dis_closest_o,player1.0_ang_closest_o,player1.0_min_dist,player1.0_ang_min,player1.0_2_min_dist,...,player4.0_s,player4.0_dist_ball,player4.0_ang_ball,player4.0_dis_closest_o,player4.0_ang_closest_o,player4.0_min_dist,player4.0_ang_min,player4.0_2_min_dist,player4.0_2_ang_min,player4.0_a
0,69.16,12.47,0,32.03,4.89,6.020797,158.976154,6.744034,169.923120,7.573407,...,2.53,8.534149,12.244807,4.812411,55.583454,0.863481,13.392498,1.193692,74.450196,3.56
1,69.01,12.32,1,32.51,5.16,6.171564,157.215882,6.625896,171.319582,7.601349,...,2.72,8.578543,13.825658,5.165578,59.264512,0.802808,13.690050,1.206192,78.037949,2.49
2,68.82,12.13,2,33.01,5.41,6.287329,155.473084,6.478349,172.995652,7.502993,...,2.89,8.618144,15.616638,5.564288,62.375742,0.764003,14.400021,1.236285,80.690060,1.98
3,68.60,11.91,3,33.53,5.73,6.326642,174.739978,6.377366,153.756380,7.363763,...,4.75,8.507203,320.772196,4.808752,273.457417,1.154340,328.099843,2.703794,288.770008,2.09
4,68.34,11.65,4,34.05,6.01,6.130000,176.726846,6.424733,151.759645,7.186167,...,4.82,8.214098,319.542455,4.981696,271.495333,1.032279,324.462322,2.364783,284.447591,2.28


In [89]:
# Model has the following inputs:
#           - 'max_distance' (distance of ball carrier from current position to final position), 'frame_since_bc' (# frames since ball carrier got the ball)
#           - ball carrier's: 'y' (position between sidelines), 's' (speed), 'a' (acceleration), distance and angle to 3 closest offensive players besides the QB
#           - 3 nearest defender's: 's', 'a', distance and angle to 1) closest offensive player to the ball carrier and 2), 3) two other nearest offensive players to defender besides QB

from sklearn.model_selection import train_test_split

X = df_reg.drop(columns=['actual_dist_from_final'])  # Features
y = df_reg['actual_dist_from_final']  # Target

# Split into training and temporary test set (which will be further split into validation and test sets)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)  # 70% train, 30% temp

# Further split the temporary test set into validation and test sets
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)  # Split the 30% into two parts of 15% each

# for week 7-9 analysis
X_wks_7_9 = df_reg_wks_7_9.drop(columns=['actual_dist_from_final'])
y_wks_7_9 = df_reg_wks_7_9['actual_dist_from_final']

In [99]:
# experimented with many other combinations


param_grid = {
    'n_estimators': [500],
    'learning_rate': [0.05],
    'max_depth': [12],
    'subsample': [1.0],
    'colsample_bytree': [0.9],
    'gamma': [0.4]

}

In [100]:
model = xgb.XGBRegressor()

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1, verbose=1)

In [101]:
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, m...
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None,
                                    random_state=None, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.9], 'gamma': [0.4],
                         'learning_rate': [0.05, 0.15], 'max_depth': [12, 20],
                         'n_estimators': [100, 500], 'subsample': [1.0]},
             scoring='neg_mean_squared_error', verbose=1)

In [102]:
best_parameters = grid_search.best_params_
best_model = grid_search.best_estimator_

print("Best Parameters:", best_parameters)

Best Parameters: {'colsample_bytree': 0.9, 'gamma': 0.4, 'learning_rate': 0.05, 'max_depth': 12, 'n_estimators': 500, 'subsample': 1.0}


In [103]:
from sklearn.metrics import mean_squared_error

y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

Mean Squared Error: 15.29827125368342


In [104]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    non_zero_mask = y_true != 0
    return np.mean(np.abs((y_true[non_zero_mask] - y_pred[non_zero_mask]) / y_true[non_zero_mask])) * 100

mape = mean_absolute_percentage_error(y_test, y_pred)

print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"R²: {r2}")
print(f"MAPE: {mape}")

MSE: 15.29827125368342
RMSE: 3.9113004555625
MAE: 1.9582301797578605
R²: 0.6904427490885439
MAPE: 139.67394398521765


In [105]:
# Create a DataFrame for the metrics
metrics_df = pd.DataFrame({
    'Metric': ['MSE', 'RMSE', 'MAE', 'MAPE', 'R²'],
    'Value': [mse, rmse, mae, mape, r2]
})

print(metrics_df)

  Metric       Value
0    MSE   15.298271
1   RMSE    3.911300
2    MAE    1.958230
3   MAPE  139.673944
4     R²    0.690443


In [23]:
!pip install plotly

In [106]:
import plotly.express as px

# Create a DataFrame for actual vs predicted values
results_df = pd.DataFrame({
    'Actual': y_test,
    'Predicted': y_pred
})

# Create a scatter plot
fig = px.scatter(results_df, x='Actual', y='Predicted', labels={'x': 'Actual Distance', 'y': 'Predicted Distance'},
                 title='Actual vs Predicted Distance')
fig.add_scatter(x=[results_df['Actual'].min(), results_df['Actual'].max()],
                y=[results_df['Actual'].min(), results_df['Actual'].max()],
                mode='lines', showlegend=False)

# Show the plot
fig.show()


In [107]:
y_pred_7_9 = best_model.predict(X_wks_7_9)
mse_7_9 = mean_squared_error(y_wks_7_9, y_pred_7_9)
print("Mean Squared Error:", mse_7_9)

Mean Squared Error: 29.085453600929572


In [108]:
mse_7_9 = mean_squared_error(y_wks_7_9, y_pred_7_9)
rmse_7_9 = mean_squared_error(y_wks_7_9, y_pred_7_9, squared=False)
mae_7_9 = mean_absolute_error(y_wks_7_9, y_pred_7_9)
r2_7_9 = r2_score(y_wks_7_9, y_pred_7_9)


mape_7_9 = mean_absolute_percentage_error(y_wks_7_9, y_pred_7_9)

print(f"MSE_7_9: {mse_7_9}")
print(f"RMSE_7_9: {rmse_7_9}")
print(f"MAE_7_9: {mae_7_9}")
print(f"R²_7_9: {r2_7_9}")
print(f"MAPE_7_9: {mape_7_9}")

MSE_7_9: 29.085453600929572
RMSE_7_9: 5.393093138536509
MAE_7_9: 2.8117570202069544
R²_7_9: 0.39600509241095105
MAPE_7_9: 189.40850176354763


In [109]:
# Create a DataFrame for actual vs predicted values
results_df = pd.DataFrame({
    'Actual': y_wks_7_9,
    'Predicted': y_pred_7_9
})

# Create a scatter plot
fig = px.scatter(results_df, x='Actual', y='Predicted', labels={'x': 'Actual Distance', 'y': 'Predicted Distance'},
                 title='Actual vs Predicted Distance')
fig.add_scatter(x=[results_df['Actual'].min(), results_df['Actual'].max()],
                y=[results_df['Actual'].min(), results_df['Actual'].max()],
                mode='lines', showlegend=False)

# Show the plot
fig.show()


In [ ]:
import shap
import numpy as np

# Initialize the SHAP explainer (assuming a deep learning model)
explainer = shap.TreeExplainer(best_model)

batch_size = 1000
shap_values_all = []

for i in range(0, len(X), batch_size):
    batch = X[i:i + batch_size].values
    shap_values_batch = explainer.shap_values(batch)
    shap_values_all.append(shap_values_batch[0])

shap_values_concatenated = np.concatenate(shap_values_all, axis=0)

[16:53:39] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.


In [ ]:
# Calculate SHAP for test data
shap_values_test = explainer.shap_values(X_wks_7_9)

In [ ]:
#pd.DataFrame(shap_values_test).to_csv('shap_values_test.csv')